# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
import json
import sys           # Ya viene con Python
import time          # Ya viene con Python
try:
    from IPython.display import Markdown, display,clear_output
    from io import StringIO
    IN_JUPYTER = True
except ImportError:
    IN_JUPYTER = False


In [52]:
# constantes

OLLAMA_API = "http://132.248.32.20:8080/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"
MODEL = "gemma3:27b"

In [53]:
system_prompt="""
            Asume que eres un tutor experto en lenguaje Python, da explicaciones 
            detalladas y con ejemplos para que el alumno comprenda 
              """

In [54]:
# Historial global
messages = []

def add_message(role, content, reset=False):
    """
    Agrega un mensaje al historial. Puede reiniciar la conversación si se indica.

    Args:
        role (str): 'system', 'user' o 'assistant'
        content (str): texto del mensaje
        reset (bool): si True, reinicia el historial antes de agregar
    """
    global messages
    if reset:
        messages = []
    if role not in ['system', 'user', 'assistant']:
        raise ValueError("El rol debe ser 'system', 'user' o 'assistant'")
    messages.append({"role": role, "content": content})


In [55]:
def chat_with_model():
    """
    Envía el historial de mensajes actual al modelo.

    Returns:
        str: respuesta del modelo
    """
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    return response.json()['message']['content']

In [56]:
def chat_with_model_stream():
    """
    Envía el historial al modelo con stream=True y muestra salida tipo máquina de escribir.
    También devuelve y muestra el resultado completo en formato JSON.
    """
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": True
    }

    with requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True) as response:
        response.raise_for_status()
        full_text = ""

        for line in response.iter_lines(decode_unicode=True):
            if not line:
                continue
            # Cada línea en streaming es un JSON (por chunk)
            chunk = json.loads(line)
            delta = chunk.get("message", {}).get("content", "")
            full_text += delta
            for char in delta:
                sys.stdout.write(char)
                sys.stdout.flush()
                time.sleep(0.01)  # velocidad de "máquina de escribir"

        print("\n\n🧾 Resultado final en JSON:")
        json_result = {
            "model": MODEL,
            "response": full_text
        }
        print(json.dumps(json_result, ensure_ascii=False, indent=2))

In [60]:
def chat_with_model_stream_markdown(save_to_file=False):
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": True
    }

    with requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True) as response:
        response.raise_for_status()
        full_text = ""

        print("🧠 Generando respuesta...\n")

        if IN_JUPYTER:
            stream_buffer = StringIO()

        for line in response.iter_lines(decode_unicode=True):
            if not line:
                continue
            chunk = json.loads(line)
            delta = chunk.get("message", {}).get("content", "")
            full_text += delta

            if IN_JUPYTER:
                stream_buffer.write(delta)
                clear_output(wait=True)
                display(Markdown(stream_buffer.getvalue()))
            else:
                sys.stdout.write(delta)
                sys.stdout.flush()
                time.sleep(0.0005)

        print("\n\n✅ Respuesta completa recibida.\n")

        if save_to_file:
            with open("respuesta.md", "w", encoding="utf-8") as f:
                f.write(full_text)
            print("📁 Markdown guardado en 'respuesta.md'")

        if not IN_JUPYTER:
            print("📝 Markdown crudo (modo consola):\n")
            #print(full_text)

        return full_text

In [62]:
#response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
#print(response.json()['message']['content'])
add_message("system", "Eres un sabio filósofo griego que habla en parábolas.", reset=True)
add_message("user", "¿Qué es la verdad?")

# Envía la conversación al modelo
respuesta = chat_with_model_stream_markdown()


Ah, joven buscador. Me preguntas por la Verdad, un fantasma que todos perseguimos, pero pocos tocan. Permíteme contarte una parábola.

Imagínate una cueva oscura. En su interior, viven prisioneros encadenados, mirando fijamente una pared. Detrás de ellos, arde un fuego, y entre el fuego y los prisioneros, pasan objetos, proyectando sombras en la pared que contemplan.

Para estos prisioneros, las sombras *son* la realidad. No conocen nada más. Si uno de ellos se liberara y obligara a mirar más allá, le dolerían los ojos al principio, cegado por la luz. Le costaría creer que lo que veía era más real que las sombras que conocía. Y si intentara regresar a contarles a sus compañeros lo que ha visto, le reirían en la cara, pensando que está loco.

La verdad, joven, no es una simple afirmación, sino un camino. Es el proceso de liberarse de las cadenas de la percepción limitada, de cuestionar las sombras que nos presentan como realidad. Es el esfuerzo por ver la fuente de la luz, por comprender la forma original de las cosas.

No es algo que se pueda *tener*, sino algo que se debe *buscar*, y la búsqueda, a menudo, es dolorosa. Porque implica abandonar la comodidad de lo conocido, para adentrarse en la incertidumbre.

Así que no me preguntes *qué es* la verdad, sino *cómo* buscarla. Busca la luz, cuestiona las sombras, y no temas al dolor de la liberación. Porque solo así, quizás, podrás vislumbrar un atisbo de su belleza.




✅ Respuesta completa recibida.



In [63]:
question = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""
add_message("system", system_prompt, reset=True)
add_message("user", question)

# Envía la conversación al modelo
IN_JUPYTER = True
respuesta = chat_with_model_stream_markdown()


¡Claro! Vamos a desglosar este código Python, que utiliza la palabra clave `yield from`. Es una construcción un poco avanzada, pero muy útil para trabajar con generadores.

**Contexto: Generadores en Python**

Antes de entrar en el código específico, es crucial entender qué son los generadores. En Python, los generadores son funciones especiales que "generan" una secuencia de valores en lugar de devolver una lista completa de una vez. Esto es muy eficiente en términos de memoria, especialmente cuando se trabaja con grandes conjuntos de datos.

La palabra clave `yield` es lo que convierte una función regular en un generador. Cuando una función encuentra `yield`, devuelve el valor especificado y "pausa" su ejecución. La próxima vez que se pida un valor del generador, reanuda la ejecución desde donde se quedó.

**Análisis del Código: `yield from {book.get("author") for book in books if book.get("author")}`**

Vamos a desglosarlo parte por parte:

1. **`{book.get("author") for book in books if book.get("author")}`:  Comprensión de Conjuntos (Set Comprehension)**

   * **`books`**: Asumimos que `books` es una lista (o cualquier iterable) de diccionarios. Cada diccionario representa un libro y probablemente tiene claves como "title", "author", etc.
   * **`book.get("author")`**:  Este código intenta obtener el valor de la clave "author" de cada diccionario `book`.  El método `.get()` es importante aquí porque:
     * Si el diccionario `book` *tiene* una clave "author", devuelve el valor asociado a esa clave (el nombre del autor).
     * Si el diccionario `book` *no tiene* una clave "author", devuelve `None` (por defecto). Esto evita que el código falle con un error `KeyError`.
   * **`if book.get("author")`**:  Esta es una condición que se aplica a cada libro.  Solo si `book.get("author")` devuelve un valor que se evalúa como `True` (es decir, un nombre de autor no vacío), se incluye ese autor en el conjunto resultante.  Esto filtra los libros que no tienen información del autor.
   * **`{... for book in books if ...}`**: Esta es una *comprensión de conjuntos*.  Es una forma concisa de crear un conjunto (una colección de elementos únicos) a partir de una secuencia (en este caso, la lista `books`). La comprensión de conjuntos itera sobre `books`, aplica la condición `if`, y crea un conjunto que contiene solo los nombres de los autores que cumplen la condición.  Un conjunto asegura que no haya autores duplicados.

2. **`yield from ...`**

   * **`yield from iterable`**: Esta es la parte clave.  `yield from` es una forma de delegar a otro generador o iterable. En este caso, está delegando a la comprensión de conjuntos que acabamos de analizar.

**¿Qué hace el código en general?**

El código hace lo siguiente:

1. **Extrae los autores de una lista de libros:** Itera sobre una lista de diccionarios (libros) y extrae el nombre del autor de cada libro, siempre y cuando el libro tenga la clave "author".
2. **Elimina autores duplicados:** Utiliza una comprensión de conjuntos para asegurarse de que solo haya un nombre de autor único en el resultado.
3. **Genera los nombres de los autores:** Utiliza `yield from` para "aplanar" el conjunto resultante y generar cada nombre de autor individualmente.  En otras palabras, transforma el conjunto en un generador.

**Ejemplo:**

```python
books = [
    {"title": "El Señor de los Anillos", "author": "J.R.R. Tolkien"},
    {"title": "1984", "author": "George Orwell"},
    {"title": "Orgullo y Prejuicio", "author": "Jane Austen"},
    {"title": "El Hobbit"},  # Sin autor
    {"title": "1984", "author": "George Orwell"},  # Duplicado
]

def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Usar el generador
for author in get_authors(books):
    print(author)
```

**Salida:**

```
J.R.R. Tolkien
George Orwell
Jane Austen
```

**¿Por qué usar `yield from` en lugar de un simple `for` loop?**

Aunque en este ejemplo sencillo podría usarse un bucle `for`, `yield from` ofrece varias ventajas:

* **Concisidad:** Hace que el código sea más legible y compacto.
* **Eficiencia:**  `yield from` está optimizado para trabajar con generadores y puede ser más eficiente en algunos casos, especialmente cuando el iterable que se delega es un generador en sí mismo.
* **Manejo de subgeneradores:**  `yield from` es especialmente útil cuando se trabaja con funciones generadoras que llaman a otras funciones generadoras. Permite delegar directamente a la subfunción generadora sin necesidad de bucles explícitos.

**En resumen:**

`yield from` es una herramienta poderosa para escribir generadores eficientes y legibles en Python.  En este caso, delega la tarea de generar nombres de autores únicos a una comprensión de conjuntos, creando un generador que produce cada nombre de autor de forma individual.




✅ Respuesta completa recibida.

